In [1]:
import pandas as pd
import numpy as np

In [2]:
calendario = pd.read_csv("data/daily_calendar_with_events.csv")
precios = pd.read_csv("data/item_prices.csv")
ventas = pd.read_csv("data/item_sales.csv")

In [60]:
ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


In [3]:
ventas1=ventas.copy()

In [4]:
ventasmelt=ventas.melt(id_vars=ventas.columns[:7],value_vars=ventas.columns[7:],var_name="d",value_name="Cantidad") #Transformacion de BD, dias por Fila

In [59]:
ventasmelt.shape

(58327370, 9)

In [5]:
ventasmelt["Cantidad"].sum() # suma de ventas totales

65695409

In [6]:
ventas2=ventasmelt.drop(ventasmelt[ventasmelt["Cantidad"]==0].index) #eliminar indices con ventas =0

In [7]:
ventasmelt[ventasmelt["Cantidad"]==0].index

Index([       0,        1,        2,        3,        4,        5,        6,
              9,       10,       11,
       ...
       58327349, 58327350, 58327351, 58327355, 58327357, 58327358, 58327363,
       58327366, 58327367, 58327369],
      dtype='int64', length=39777094)

In [8]:
ventas2.reset_index(drop= True,inplace=True) # reset de indices despues de eliminar

In [9]:
ventas_totales=ventas2["Cantidad"].sum() # comprobacion de suma total despues de eliminar filas check

In [10]:
calendario2=calendario.copy()

In [11]:
calendario2["date"]=pd.to_datetime(calendario2["date"]) # formato fecha en columna date

In [12]:
# crea columnas de dia,semana,mes y año a partir de date
calendario2['week'] = calendario2['date'].dt.isocalendar().week
calendario2['day'] = calendario2['date'].dt.day
calendario2['month'] = calendario2['date'].dt.month
calendario2['year'] = calendario2['date'].dt.year

In [13]:
# concatena semana y año en un string, agregando el 0 a semanas del 1 al 9
calendario2['week'] = calendario2['date'].dt.isocalendar().week.astype(str)
for i in range(len(calendario2)):
    if int(calendario2.loc[i,'week']) < 10:
        calendario2.loc[i,'week'] = '0'+calendario2.loc[i,'week']
calendario2['year'] = calendario2['date'].dt.isocalendar().year
calendario2['yearweek']=calendario2['year'].astype(str)+calendario2['week']

In [14]:
ventas3=pd.merge(ventas2,calendario2[["d","weekday_int","event","month","day","yearweek"]],how="left",on="d") # Join ventas con calendario

In [15]:
ventas3.shape

(18550276, 14)

In [16]:
ventas3.head(1)

,id,item,category,department,store,store_code,region,d,Cantidad,weekday_int,event,month,day,yearweek
0,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,12,1,NaN,1,29,201104


In [17]:
columnasbase=ventas3.columns[:7].tolist()
columnasbase.append("yearweek")
columnasbase

['id',
 'item',
 'category',
 'department',
 'store',
 'store_code',
 'region',
 'yearweek']

In [18]:
ventas4=ventas3.groupby(by=columnasbase).agg({"Cantidad":"sum"})

In [19]:
ventas4=ventas4.reset_index()

In [20]:
ventas4["Cantidad"].sum()

65695409

In [21]:
ventas4.shape

(4990898, 9)

In [22]:
precios2=precios.copy()

In [23]:
precios["yearweek"].isnull().sum() # numero de semanas nulas

243920

In [24]:
precios["sell_price"].isnull().sum()

0

In [25]:
precios2["yearweek"]=precios2["yearweek"].astype(str)
precios2["yearweek"]=precios2["yearweek"].str.slice(stop=6) # quita .0 del string
precios2["year"]=precios2["yearweek"].str.slice(stop=4) # crea columna con año
precios2["semana"]=precios2["yearweek"].str.slice(start=4,stop=6) # crea columna con semana

In [26]:
precios2["id"]=precios2["item"]+"_"+precios2["store_code"] # crea id coditem+codstore

In [27]:
ventas4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4990898 entries, 0 to 4990897
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          object
 1   item        object
 2   category    object
 3   department  object
 4   store       object
 5   store_code  object
 6   region      object
 7   yearweek    object
 8   Cantidad    int64 
dtypes: int64(1), object(8)
memory usage: 342.7+ MB


In [28]:
ventas5=pd.merge(ventas4,precios2[["id","yearweek","sell_price"]],how="left",on=["id","yearweek"]) #join de ventas y precios

In [29]:
ventas5.shape

(4990898, 10)

In [30]:
ventas5[ventas5["sell_price"].isnull()].index # ventas con datos nulos dado a que le semanaaño era nula

Index([     99,     231,     360,     498,     775,     917,     999,    1165,
          1866,    2223,
       ...
       4989876, 4990115, 4990211, 4990279, 4990373, 4990433, 4990506, 4990598,
       4990714, 4990774],
      dtype='int64', length=35126)

In [31]:
ventas_nulas=ventas5["Cantidad"].iloc[ventas5[ventas5["sell_price"].isnull()].index].sum() # cuantifica la cantidad de unidades con precio nulo

In [32]:
ventas_nulas

335665

In [33]:
ventas_totales

65695409

In [34]:
ventassinprecio=ventas_totales-ventas_nulas
ventassinprecio

65359744

In [35]:
print((ventas_nulas/ventas_totales)*100,"%" )  #porcentaje de ventas nulas

0.510941335337451 %


In [36]:
from pathlib import Path  
filepath = Path("data/ventassem.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
ventas5.to_csv(filepath,index=False)  

In [37]:
calendario2.head()

,date,weekday,weekday_int,d,event,week,day,month,year,yearweek
0,2011-01-29,Saturday,1,d_1,NaN,04,29,1,2011,201104
1,2011-01-30,Sunday,2,d_2,NaN,04,30,1,2011,201104
2,2011-01-31,Monday,3,d_3,NaN,05,31,1,2011,201105
3,2011-02-01,Tuesday,4,d_4,NaN,05,1,2,2011,201105
4,2011-02-02,Wednesday,5,d_5,NaN,05,2,2,2011,201105


In [38]:
len(calendario2["yearweek"].unique())

274

In [39]:
from pathlib import Path  
filepath = Path("data/calendario.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
calendario2.to_csv(filepath,index=False)  

In [40]:
dep=ventas.groupby(["store_code","region","store","department"]).agg({"id":"count"})
dep=dep.reset_index()
del(dep["id"])
dep

,store_code,region,store,department
0,BOS_1,Boston,South_End,ACCESORIES_1
1,BOS_1,Boston,South_End,ACCESORIES_2
2,BOS_1,Boston,South_End,HOME_&_GARDEN_1
3,BOS_1,Boston,South_End,HOME_&_GARDEN_2
4,BOS_1,Boston,South_End,SUPERMARKET_1
...,...,...,...,...
65,PHI_3,Philadelphia,Queen_Village,HOME_&_GARDEN_1
66,PHI_3,Philadelphia,Queen_Village,HOME_&_GARDEN_2
67,PHI_3,Philadelphia,Queen_Village,SUPERMARKET_1
68,PHI_3,Philadelphia,Queen_Village,SUPERMARKET_2


In [41]:
from pathlib import Path  
filepath = Path("data/depto.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
dep.to_csv(filepath,index=False) 

In [42]:
prod=ventas.groupby(["category","item"]).agg({"id":"count"})
prod=prod.reset_index()
del(prod["id"])
prod

,category,item
0,ACCESORIES,ACCESORIES_1_001
1,ACCESORIES,ACCESORIES_1_002
2,ACCESORIES,ACCESORIES_1_003
3,ACCESORIES,ACCESORIES_1_004
4,ACCESORIES,ACCESORIES_1_005
...,...,...
3044,SUPERMARKET,SUPERMARKET_3_823
3045,SUPERMARKET,SUPERMARKET_3_824
3046,SUPERMARKET,SUPERMARKET_3_825
3047,SUPERMARKET,SUPERMARKET_3_826


In [43]:
from pathlib import Path  
filepath = Path("data/productos.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
prod.to_csv(filepath,index=False) 

In [44]:
ventas6=ventas5.copy()

In [45]:
ventas6["sell_price"]=ventas6["sell_price"].interpolate(method='linear')

In [46]:
ventas6[ventas5["sell_price"].isnull()].index

Index([     99,     231,     360,     498,     775,     917,     999,    1165,
          1866,    2223,
       ...
       4989876, 4990115, 4990211, 4990279, 4990373, 4990433, 4990506, 4990598,
       4990714, 4990774],
      dtype='int64', length=35126)

In [47]:
ventas6.iloc[ventas6[ventas5["sell_price"].isnull()].index]

,id,item,category,department,store,store_code,region,yearweek,Cantidad,sell_price
99,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201553,4,10.9858
231,ACCESORIES_1_001_BOS_2,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Roxbury,BOS_2,Boston,201553,5,10.9858
360,ACCESORIES_1_001_BOS_3,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Back_Bay,BOS_3,Boston,201553,9,10.9858
498,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,201553,6,10.9858
775,ACCESORIES_1_001_NYC_3,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Tribeca,NYC_3,New York,201553,4,10.9858
...,...,...,...,...,...,...,...,...,...,...
4990433,SUPERMARKET_3_827_NYC_1,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Greenwich_Village,NYC_1,New York,201553,23,1.2000
4990506,SUPERMARKET_3_827_NYC_2,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Harlem,NYC_2,New York,201553,13,1.2000
4990598,SUPERMARKET_3_827_NYC_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Tribeca,NYC_3,New York,201553,41,1.2000
4990714,SUPERMARKET_3_827_PHI_1,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Midtown_Village,PHI_1,Philadelphia,201553,26,1.2000


In [53]:
#lista de productos que no se pueden interpolar

lista=[]
for i in range(len(ventas5)):
    if np.isnan(ventas5["sell_price"].iloc[i]) and ((ventas5["id"].iloc[i-1]!=ventas5["id"].iloc[i]) or (ventas5["id"].iloc[i+1]!=ventas5["id"].iloc[i])) and (ventas5["sell_price"].iloc[i+1]-ventas5["sell_price"].iloc[i-1])!=0:
        lista.append(i)
len(lista)      

9444

In [52]:
ventas5["Cantidad"].iloc[lista].sum()

62068

In [56]:
ventas_nulas2=ventas6["Cantidad"].iloc[ventas6[ventas6["sell_price"]==0].index].sum()

In [57]:
ventas_nulas2

62068

In [58]:
ventas_nulas

335665

In [ ]:
#ventas6["sell_price"].iloc[lista]=0